In [1]:
import eikon as ek
import pandas as pd
import numpy as np
import talib as ta
#import pause
#import chatbot_demo_rest as cdr
import sys,os
import logging
#from rdp_token import RDPTokenManagement
from datetime import datetime
from dateutil.relativedelta import relativedelta
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
import matplotlib.gridspec as gridspec
from matplotlib.ticker import Formatter
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

>Simple Moving Averages

This function uses the TA-Lib SMA function to calculate the Simple Moving Average
using the Close price for two periods 
which you will note later are 14 for the short period and 200 for the long period.
As you will see later, the period interval itself can vary 
e.g minute, daily, monthly, hourly - so for example, calculate SMA for 14 days and 200 days.

In [2]:
def SMA(close,sPeriod,lPeriod):
    shortSMA = ta.SMA(close,sPeriod)
    longSMA = ta.SMA(close,lPeriod)
    smaSell = ((shortSMA <= longSMA) & (shortSMA.shift(1) >= longSMA.shift(1)))
    smaBuy = ((shortSMA >= longSMA) & (shortSMA.shift(1) <= longSMA.shift(1)))
    return smaSell,smaBuy,shortSMA,longSMA

>Relative Strength Index

RSI calculation is usually done for a 14 day period - so once again I feed in the Close price for the instrument to the TA-Lib RSI function. The common methodology is to set high and low thresholds of the RSI at 70 and 30. The idea is that if the lower threshold is crossed, the asset is becoming oversold and we should buy. Conversely, if the upper threshold is crossed then the asset is becoming overbought and we should sell.

In [ ]:
def RSI(close,timePeriod):
    rsi = ta.RSI(close,timePeriod)
    rsiSell = (rsi>70) & (rsi.shift(1)<=70)
    rsiBuy = (rsi<30) & (rsi.shift(1)>=30)
    return rsiSell,rsiBuy, rsi

>Stochastics

The TA-Lib Stoch function returns two lines slowk and slowd which can then be used to generate the buy/sell indicators. A crossover signal occurs when the two lines cross in the overbought region (commonly above 80) or oversold region (commonly below 20). When a slowk line crosses below the slowd line in the overbought region it is considered a sell indicator. Conversely, when an increasing slowk line crosses above the slowd line in the oversold region it is considered a buy indicator.

In [3]:
def Stoch(close,high,low):
    slowk, slowd = ta.STOCH(high, low, close)
    stochSell = ((slowk < slowd) & (slowk.shift(1) > slowd.shift(1))) & (slowd > 80)
    stochBuy = ((slowk > slowd) & (slowk.shift(1) < slowd.shift(1))) & (slowd < 20)
    return stochSell,stochBuy, slowk,slowd


>We have a Trade Signal - so send it to the Chat Room!

I need a way of letting users know that a Trade signal has been generated by the Technical Analysis. So, I am going to use the Messenger BOT API to send messages to other Refinitiv Messenger users. I am re-purposing the existing MessengerChatBot.Python example from GitHub.

In [ ]:
# Key code snippets - see Github for full source
def sendSignaltoChatBot(myRIC, signalTime, indicators):
    indicatorList = ','.join(indicators.values)
    message = f"TA signal(s) Generated : {indicatorList} at {signalTime} for {myRIC}"
    # Connect, login and send message to chatbot
    rdp_token = RDPTokenManagement( bot_username, bot_password, app_key)
    access_token = cdr.authen_rdp(rdp_token)
    if access_token:
        # Join associated Chatroom
        joined_rooms = cdr.join_chatroom(access_token, chatroom_id)
        if joined_rooms:
            cdr.post_message_to_chatroom(access_token, joined_rooms, chatroom_id, message)

>Run the Technical Analysis

Initially, I will do a historical TA run, after which I will use this function to run the above 3 TA methodologies on the data I get as part of the ongoing live Technical Analysis.

I am going to repeat some of this code later in the main historical TA run loop - purely for ease of reading.

In [ ]:
def runAllTA(myRIC, data):
    
    price = data['CLOSE']
    high = data['HIGH']
    low = data['LOW']
        
    # Simple Moving Average calcs
    smaSell,smaBuy,shortSMA,longSMA = SMA(price,shortPeriod,longPeriod)
    # Do the RSI calcs
    rsiSell,rsiBuy,rsi = RSI(price,shortPeriod)
    # and now the stochastics
    stochSell,stochBuy,slowk,slowd = Stoch(price, high, low)    
    # Now collect buy and sell Signal timestamps into a single df
    sigTimeStamps = pd.concat([smaSell, smaBuy, stochSell, stochBuy, rsiSell, rsiBuy],axis=1)
    sigTimeStamps.columns=['SMA Sell','SMA Buy','Stoch Sell','Stoch Buy','RSI Sell','RSI Buy']
    signals = sigTimeStamps.loc[sigTimeStamps['SMA Sell'] | sigTimeStamps['Stoch Sell'] |
                         sigTimeStamps['RSI Sell'] | sigTimeStamps['SMA Buy'] |
                         sigTimeStamps['Stoch Buy'] | sigTimeStamps['RSI Buy']]
    
    # Compare final signal Timestamp with latest data TimeStamp
    if (data.index[-1]==signals.index[-1]):
        final = signals.iloc[-1]
        # filter out the signals set to True and send to ChatBot
        signal = final.loc[final]
        signalTime = signal.name.strftime("%Y-%m-%dT%H:%M:%S")
        indicators = signal.loc[signal].index
        sendSignaltoChatBot(myRIC, signalTime, indicators)

> Timing Helper functions

In [4]:
# Calculate Start and End time for our historical data request window
def startEnd(interval):
    end = datetime.now()
    start = {
      'minute': lambda end: end - relativedelta(days=5),
      'hour': lambda end: end - relativedelta(months=2),
      'daily': lambda end: end - relativedelta(years=2),
      'weekly': lambda end: end - relativedelta(years=5),
      'monthly': lambda end: end - relativedelta(years=10),
    }[interval](end)
    return start.strftime("%Y-%m-%dT%H:%M:%S"),end.strftime("%Y-%m-%dT%H:%M:%S")

> Plotting functions

Whilst not essential to the workflow, I wanted to plot a few charts to provide a visual representation of the various TA indicators - so we can try and visually tie-up instances where a price rises or drops in line with a TA trade signal - so for example when the short SMA crosses up through the long SMA, do we see an upward trend in the price after that point in time?

In [5]:
# As per before, key code snips only...
# Use a formatter to remove weekends from date axis
# to smooth out the line.
class MyFormatter(Formatter):
    def __init__(self, dates, fmt='%Y-%m-%d'):
        self.dates = dates
        self.fmt = fmt
    def __call__(self, x, pos=0):
        'Return the label for time x at position pos'
        ind = int(round(x))
        if ind>=len(self.dates) or ind<0: return ''
        return self.dates[ind].strftime(self.fmt)
# Plot the Close price and short and long Simple Moving Averages
def plotSMAs(ric,close,sma14,sma200,sell,buy):
    x = close.index
    plt.rcParams["figure.figsize"] = (28,8)
    fig, ax = plt.subplots(facecolor='0.25')
    ax.plot(np.arange(len(x)),close, label='Close',color='y')
    ax.plot(np.arange(len(x)),sma14,label="SMA 14", color='g')
    ax.plot(np.arange(len(x)),sma200,label="SMA 200", color='tab:purple')
    plt.show()
# Plot the Close price in the top chart and RSI in the lower chart
def plotRSI(ric,close,rsi):
    plt.rcParams["figure.figsize"] = (28,12)
    fig = plt.figure(facecolor='0.25')
    gs1 = gridspec.GridSpec(2, 1)
    # RSI chart
    ax = fig.add_subplot(gs1[1])
    ax.xaxis.set_major_formatter(formatter)
    ax.plot(np.arange(len(rsi.index)), rsi.values,color='b')
    plt.axhline(y=70, color='w',linestyle='--')
    plt.axhline(y=30, color='w',linestyle='--')
    # Close Price chart
    axc = fig.add_subplot(gs1[0])
    axc.plot(np.arange(len(rsi.index)), close, color='y')
    plt.show()
# Plot Close price in top chart and in the slowk + slowd lines in lower chart
def plotStoch(ric,close,slowK,slowD):
    plt.rcParams["figure.figsize"] = (28,12)
    fig = plt.figure(facecolor='0.25')
    gs1 = gridspec.GridSpec(2, 1)
    ax = fig.add_subplot(gs1[1])
    # Stochastic lines chart
    ax.plot(np.arange(len(slowk.index)), slowk.values,label="Slow K",color='m')
    ax.plot(np.arange(len(slowk.index)), slowd.values,label="Slow D",color='g')
    plt.axhline(y=80, color='w',linestyle='--')
    plt.axhline(y=20, color='w',linestyle='--')
    # Closing price chart
    axc = fig.add_subplot(gs1[0])
    axc.plot(np.arange(len(close.index)), close, color='y')
    plt.show()

> Connecting to the Eikon application


To connect to my running instance of Eikon (or Workspace) I need to provide my Application Key.

In [6]:
ek.__version__
ek.set_app_key('<your app key>')

2023-04-03 02:02:13,868 P[10560] [MainThread 8364] Error: no proxy address identified.
Check if Eikon Desktop or Eikon API Proxy is running.
2023-04-03 02:02:13,868 P[10560] [MainThread 8364] Error on handshake url http://127.0.0.1:None/api/handshake : UnsupportedProtocol("Request URL is missing an 'http://' or 'https://' protocol.")
2023-04-03 02:02:13,876 P[10560] [MainThread 8364] Error on handshake url http://127.0.0.1:None/api/handshake : UnsupportedProtocol("Request URL is missing an 'http://' or 'https://' protocol.")
2023-04-03 02:02:13,876 P[10560] [MainThread 8364] Port number was not identified, cannot send any request


> initialisation code

I need to calculate the start and end date for my price query - based on the chosen periodicity/interval, as well as specify the periods for moving averages.

Also, as I will be requesting the price of each instrument individually, I create a container to hold all the price data for the full basket of instruments.

Finally, I set some display properties for the Pandas dataframe.

In [ ]:
myInterval = 'daily'    # 'minute', 'hour', 'daily', 'weekly', 'monthly'
myStart, myEnd  = startEnd(myInterval)
timestampLen = timeStampLength(myInterval)
print(f'Interval {myInterval} from {myStart} to {myEnd} : Timestamp Length {timestampLen}')
shortPeriod = 14
longPeriod = 200
basket={}
# Do we want to plot charts?
plotCharts = True
# Dataframe display setting
pd.set_option("display.max_rows", 999)
pd.set_option('precision', 3)

> TA Analysis summary output

Once the initial historical TA has been run, I want to present a summary table of the signal over that period.

For this, I am going to use a Dataframe to output the results in a readable format.

I am also creating some blank columns which I will use for padding the dataframe later.

In [8]:
outputDF = pd.DataFrame(columns=['RIC','Name','ISIN','Close','Periodicity','Intervals Up',
                               'Intervals Down','Unchanged','1wk %ch','1M %ch','YTD %ch','6M %ch',
                               '1yr %ch','SMA Sell','SMA Buy','Stoch Sell','Stoch Buy','RSI Sell', 'RSI Buy' ])
blankSignalCols = ['N/A']*6
blankNonSignalCols = [' ']*13